<a href="https://colab.research.google.com/github/BilalKhaliqWillis/BILAL-Assignment2/blob/main/BILAL_Assignment_14_Customizing_Deep_Neural_Network_(DNN)_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Assignment 14: Customizing Deep Neural Network (DNN) Training
# Importing Required Libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# Loading the California Housing Dataset
# Loading dataset
housing = fetch_california_housing()

X = housing.data
y = housing.target

print("Feature shape:", X.shape)
print("Target shape:", y.shape)

Feature shape: (20640, 8)
Target shape: (20640,)


In [4]:
# Splitting the Dataset (Train / Validation / Test)
# First split:- Train + Temp
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Second split:- Train / Validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42
)

print("Training set:", X_train.shape)
print("Validation set:", X_val.shape)
print("Test set:", X_test.shape)

Training set: (13209, 8)
Validation set: (3303, 8)
Test set: (4128, 8)


In [5]:
# Standardizing the Features
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [6]:
# Building the Sequential Model with L2 Regularization
model = keras.Sequential([
    layers.Dense(
        30,
        activation="relu",
        kernel_regularizer=regularizers.l2(0.05),
        input_shape=X_train_scaled.shape[1:]
    ),
    layers.Dense(1)
])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 30)             │           270 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 301 (1.18 KB)

 Trainable params: 301 (1.18 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Defining the Loss Function, Optimizer and Metrics
loss_fn = keras.losses.MeanSquaredError()

optimizer = keras.optimizers.SGD(learning_rate=0.01)

train_mae = keras.metrics.MeanAbsoluteError()
val_mae = keras.metrics.MeanAbsoluteError()

In [8]:
# Preparing the Dataset Using tf.data
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices(
    (X_train_scaled, y_train)
).shuffle(1000).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices(
    (X_val_scaled, y_val)
).batch(batch_size)

In [9]:
# Custom Training Loop Using GradientTape
epochs = 10

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")

    # Reset metrics
    train_mae.reset_state()
    val_mae.reset_state()

    # Training loop
    for X_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            predictions = model(X_batch, training=True)
            loss = loss_fn(y_batch, predictions)
            loss += sum(model.losses)  # L2 regularization loss

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_mae.update_state(y_batch, predictions)

    # Validation loop
    for X_batch, y_batch in val_dataset:
        val_predictions = model(X_batch, training=False)
        val_mae.update_state(y_batch, val_predictions)

    print(f"Train MAE: {train_mae.result():.4f}")
    print(f"Validation MAE: {val_mae.result():.4f}")


Epoch 1/10
Train MAE: 0.5990
Validation MAE: 0.4868

Epoch 2/10
Train MAE: 0.4683
Validation MAE: 0.4878

Epoch 3/10
Train MAE: 0.4645
Validation MAE: 0.4831

Epoch 4/10
Train MAE: 0.4641
Validation MAE: 0.4812

Epoch 5/10
Train MAE: 0.4628
Validation MAE: 0.4857

Epoch 6/10
Train MAE: 0.4628
Validation MAE: 0.4906

Epoch 7/10
Train MAE: 0.4619
Validation MAE: 0.4687

Epoch 8/10
Train MAE: 0.4613
Validation MAE: 0.4767

Epoch 9/10
Train MAE: 0.4608
Validation MAE: 0.4757

Epoch 10/10
Train MAE: 0.4590
Validation MAE: 0.4834


In [10]:
# Evaluating on the Test Set
test_predictions = model(X_test_scaled, training=False)
test_mae = keras.metrics.MeanAbsoluteError()
test_mae.update_state(y_test, test_predictions)

print("Test MAE:", test_mae.result().numpy())

Test MAE: 0.4747899


Answers:-
1. How do custom training loops differ from Keras’ fit() method?
- Custom training loops require manual control over forward passes, loss computation, gradient calculation and parameter updates.
The fit() method automates these steps making it simpler but less flexible.

2. Benefits and Challenges of Custom Training Loops
- Benefits:-
•	Full control over training logic
•	Easier implementation of custom losses and metrics
•	Greater flexibility for research and experimentation
- Challenges:-
•	More code to write and debug
•	Higher risk of implementation errors
•	Less beginner-friendly than fit()
